In [1]:
import os

# Path to the main dataset directory
dataset_path = 'eye_diseases_classification'

# Iterate over each subfolder in the dataset directory
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):  # Check if it's a directory
        print(f"Contents of folder '{folder}':")
        # List all files in the subfolder and print their names
        for filename in os.listdir(folder_path):
            print(filename)
        print("\n")  # Add a newline for better readability between folders

Contents of folder 'cataract':
0_left.jpg
103_left.jpg
1062_right.jpg
1083_left.jpg
1084_right.jpg
1101_right.jpg
1102_left.jpg
1102_right.jpg
1115_left.jpg
1126_right.jpg
112_right.jpg
1144_left.jpg
1144_right.jpg
1164_left.jpg
1167_right.jpg
119_left.jpg
1285_left.jpg
1415_left.jpg
1415_right.jpg
1435_left.jpg
1444_left.jpg
1452_left.jpg
1454_right.jpg
1456_left.jpg
1456_right.jpg
1460_right.jpg
1468_left.jpg
1483_left.jpg
1483_right.jpg
1484_left.jpg
1484_right.jpg
1490_right.jpg
188_right.jpg
1968_left.jpg
1987_left.jpg
1994_left.jpg
2052_left.jpg
2052_right.jpg
2074_right.jpg
2095_right.jpg
2096_right.jpg
2097_right.jpg
2098_left.jpg
2098_right.jpg
2099_left.jpg
2100_right.jpg
2101_left.jpg
2101_right.jpg
2102_left.jpg
2102_right.jpg
2103_left.jpg
2103_right.jpg
2104_left.jpg
2104_right.jpg
2105_left.jpg
2105_right.jpg
2106_right.jpg
2107_left.jpg
2108_left.jpg
2108_right.jpg
2109_left.jpg
2109_right.jpg
2110_left.jpg
2110_right.jpg
2111_left.jpg
2112_right.jpg
2113_left.jpg
2113_

In [2]:
import os
import csv

# Directory containing the dataset
base_dir = 'eye_diseases_classification'

# Prepare to write to a CSV file
csv_file = open('eye_disease_labels.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Filename', 'Eye', 'Label'])

# Loop through each disease folder
for label in ['normal', 'diabetic retinopathy', 'glaucoma', 'cataract',
              'macular degeneration', 'hypertensive', 'myopia', 'other' ]:
    folder_path = os.path.join(base_dir, label)
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            eye = 'left' if 'left' in filename else 'right'
            csv_writer.writerow([filename, eye, label])

# Close the CSV file
csv_file.close()

In [3]:
import pandas as pd
df = pd.read_csv('eye_disease_labels.csv')

In [4]:
df

,Filename,Eye,Label
0,01_h.jpg,right,normal
1,02_h.jpg,right,normal
2,03_h.jpg,right,normal
3,04_h.jpg,right,normal
4,0509201403_20140905_111423_Color_L_001.jpg,right,normal
...,...,...,...
47460,Retinitis Pigmentosa95.jpg,right,other
47461,Retinitis Pigmentosa96.jpg,right,other
47462,Retinitis Pigmentosa97.jpg,right,other
47463,Retinitis Pigmentosa98.jpg,right,other


In [22]:
import os
import pandas as pd

# Load the existing CSV file
existing_df = pd.read_csv('odir_rfmid_vietai_MURED.csv')

# Determine the starting ID by finding the max ID in the existing DataFrame
if 'ID' in existing_df.columns:
    start_id = existing_df['ID'].max() + 1
else:
    start_id = 0

# Base directory for the dataset
base_dir = 'eye_diseases_classification'

# Define a DataFrame similar to the existing one but empty for new entries
columns = ['ID', 'Patient Age', 'Patient Sex', 'Left-Fundus', 'Right-Fundus',
           'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords', 'N', 'D', 'G', 'C', 'A', 'H', 'M', 'O',
           'filepath', 'labels', 'target', 'filename', 'old_filename']
rows = []

# Dictionary to hold condition flags for simplicity
condition_flags = {'normal': 'N', 'diabetic retinopathy': 'D', 'glaucoma': 'G', 'cataract': 'C',
                   'macular degeneration':'A', 'hypertensive': 'H', 'myopia': 'M', 'other': 'O' }

# Iterate through each subfolder in the dataset directory
for folder in os.listdir(base_dir):
    condition_code = condition_flags.get(folder, '')
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            direction = 'left' if 'left' in filename else 'right'
            eye = 'Left-Fundus' if 'left' in filename else 'Right-Fundus'
            diagnostic_key = folder  # The category name
            flags = [0] * 8  # assuming 8 different condition flags as per the schema
            if condition_code:
                flags[columns.index(condition_code)+1 - 8] = 1  # set the condition flag based on folder

            # Generate new filename
            new_filename = f'{start_id}_{direction}.jpg'

            # Create the row with the next available ID
            row = {
                'ID': start_id,
                'Patient Age': 'Unknown',  # or a default/assumed value
                'Patient Sex': 'Unknown',  # or a default/assumed value
                eye: new_filename,
                'Left-Diagnostic Keywords': diagnostic_key if direction == 'left' else '',
                'Right-Diagnostic Keywords': diagnostic_key if direction == 'right' else '',
                'N': flags[0],
                'D': flags[1],
                'G': flags[2],
                'C': flags[3],
                'A': flags[4],
                'H': flags[5],
                'M': flags[6],
                'O': flags[7],
                'filepath': f'Training Images/{new_filename}',
                'labels': [folder[1].upper()] if folder == 'macular degeneration' else [folder[0].upper()],
                'target': flags,
                'filename': new_filename,
                'old_filename': filename
            }
            rows.append(row)
            start_id += 1  # Increment the ID for the next entry

# Convert the list of rows to a DataFrame
new_df = pd.DataFrame(rows, columns=columns)

# Now merge or append to the original DataFrame and save
#full_df = pd.concat([existing_df, new_df], ignore_index=True)
#full_df.to_csv('all_datasets.csv', index=False)


In [23]:
new_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename,old_filename
0,13628,Unknown,Unknown,13628_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13628_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13628_left.jpg,0_left.jpg
1,13629,Unknown,Unknown,13629_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13629_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13629_left.jpg,103_left.jpg
2,13630,Unknown,Unknown,NaN,13630_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13630_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13630_right.jpg,1062_right.jpg
3,13631,Unknown,Unknown,13631_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13631_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13631_left.jpg,1083_left.jpg
4,13632,Unknown,Unknown,NaN,13632_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13632_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13632_right.jpg,1084_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49124,62752,Unknown,Unknown,NaN,62752_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62752_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62752_right.jpg,Retinitis Pigmentosa95.jpg
49125,62753,Unknown,Unknown,NaN,62753_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62753_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62753_right.jpg,Retinitis Pigmentosa96.jpg
49126,62754,Unknown,Unknown,NaN,62754_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62754_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62754_right.jpg,Retinitis Pigmentosa97.jpg
49127,62755,Unknown,Unknown,NaN,62755_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62755_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62755_right.jpg,Retinitis Pigmentosa98.jpg


In [3]:
new_df[new_df['old_filename'].str.contains(".JPEG")]

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename,old_filename


In [18]:
import os
import shutil
# Paths to the original folders and the new folder
#training_path = 'MURED/images'
#new_images_path = 'MURED/images2'

# Create the new folder if it doesn't exist
#if not os.path.exists(new_images_path):
 #   os.makedirs(new_images_path)

def move_and_rename_images(df, original_folder, new_folder):
    for index, row in df.iterrows():
        # Attempt to find the file with both .png and .tif extensions
        #original_filename_jpg = f"{row['old_filename']}.jpg"
        #original_filename_JPG = f"{row['old_filename']}.JPG"
        #original_filename_jpeg = f"{row['old_filename']}.jpeg"
        #original_filename_png = f"{row['old_filename']}.png"
        #original_filename_TIF = f"{row['old_filename']}.TIF"
        original_filename_jpg = f"{row['old_filename']}"
        original_filename_JPG = f"{row['old_filename']}"
        original_filename_jpeg = f"{row['old_filename']}"
        original_filename_png = f"{row['old_filename']}"
        original_filename_TIF = f"{row['old_filename']}"
        original_file_path_jpg = os.path.join(original_folder, original_filename_jpg)
        original_file_path_JPG = os.path.join(original_folder, original_filename_JPG)
        original_file_path_jpeg = os.path.join(original_folder, original_filename_jpeg)
        original_file_path_png = os.path.join(original_folder, original_filename_png)
        original_file_path_TIF = os.path.join(original_folder, original_filename_TIF)
        
        if os.path.exists(original_file_path_jpg):
            new_filename = f"{row['filename']}"
            new_file_path = os.path.join(new_folder, new_filename)
            shutil.copy(original_file_path_png, new_file_path)
        elif os.path.exists(original_file_path_JPG):
            new_filename = f"{row['filename']}"
            new_file_path = os.path.join(new_folder, new_filename)
            shutil.copy(original_file_path_tif, new_file_path)
        elif os.path.exists(original_file_path_jpeg):
            new_filename = f"{row['filename']}"
            new_file_path = os.path.join(new_folder, new_filename)
            shutil.copy(original_file_path_tif, new_file_path)
        elif os.path.exists(original_file_path_png):
            new_filename = f"{row['filename']}"
            new_file_path = os.path.join(new_folder, new_filename)
            shutil.copy(original_file_path_tif, new_file_path)
        elif os.path.exists(original_file_path_TIF):
            new_filename = f"{row['filename']}"
            new_file_path = os.path.join(new_folder, new_filename)
            shutil.copy(original_file_path_tif, new_file_path)
        else:
            print(f"File not found: {original_file_path_jpg} or {original_file_path_JPG} or {original_file_path_jpeg} or {original_file_path_png} or {original_file_path_TIF}")


In [20]:
"""
normal_df = new_df[new_df['labels'].apply(lambda x: 'N' in x)]
diabetic_df = new_df[new_df['labels'].apply(lambda x: 'D' in x)]
glaucoma_df = new_df[new_df['labels'].apply(lambda x: 'G' in x)]
cataract_df = new_df[new_df['labels'].apply(lambda x: 'C' in x)]
mucular_degeneration_df = new_df[new_df['labels'].apply(lambda x: 'A' in x)]
hypertensive_df = new_df[new_df['labels'].apply(lambda x: 'H' in x)]
myopia_df = new_df[new_df['labels'].apply(lambda x: 'M' in x)]
other_df = new_df[new_df['labels'].apply(lambda x: 'O' in x)]

In [21]:
"""
normal_path = 'eye_diseases_classification/normal/'
diabetic_path = 'eye_diseases_classification/diabetic retinopathy/'
glaucoma_path = 'eye_diseases_classification/glaucoma/'
cataract_path = 'eye_diseases_classification/cataract/'
mucular_degeneration_path = 'eye_diseases_classification/macular degeneration/'
hypertensive_path = 'eye_diseases_classification/hypertensive/'
myopia_path = 'eye_diseases_classification/myopia/'
other_path = 'eye_diseases_classification/other/'

In [5]:
"""
new_images_path = 'eye_diseases_classification2'
if new_df['N'] == 1 :
    training_path = 'eye_diseases_classification/normal/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['D'] == 1 :
    training_path = 'eye_diseases_classification/diabetic retinopathy/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['G'] == 1 :
    training_path = 'eye_diseases_classification/glaucoma/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['C'] == 1 :
    training_path = 'eye_diseases_classification/cataract/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['A'] == 1 :
    training_path = 'eye_diseases_classification/macular degeneration/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['H'] == 1 :
    training_path = 'eye_diseases_classification/hypertensive/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['M'] == 1 :
    training_path = 'eye_diseases_classification/myopia/'
    move_and_rename_images(new_df, training_path, new_images_path)
if new_df['O'] == 1 :
    training_path = 'eye_diseases_classification/other/'
    move_and_rename_images(new_df, training_path, new_images_path)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
new_images_path = 'eye_diseases_classification2'
def process_images(df, disease_columns, new_images_path):
    for column in disease_columns:
        # Filter rows where the disease flag is 1
        disease_df = df[df[column] == 1]

        # Define the training path based on the disease type
        if column == 'N':
            training_path = 'eye_diseases_classification/normal/'
        elif column == 'D':
            training_path = 'eye_diseases_classification/diabetic retinopathy/'
        elif column == 'G':
            training_path = 'eye_diseases_classification/glaucoma/'
        elif column == 'C':
            training_path = 'eye_diseases_classification/cataract/'
        elif column == 'A':
            training_path = 'eye_diseases_classification/macular degeneration/'
        elif column == 'H':
            training_path = 'eye_diseases_classification/hypertensive/'
        elif column == 'M':
            training_path = 'eye_diseases_classification/myopia/'
        elif column == 'O':
            training_path = 'eye_diseases_classification/other/'

        # Execute the move_and_rename_images function if there are rows to process
        if not disease_df.empty:
            move_and_rename_images(disease_df, training_path, new_images_path)

In [20]:
# List of columns to check
disease_columns = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

# Call the processing function
process_images(new_df, disease_columns, new_images_path)

In [17]:
muscular_degeneration_df = new_df[new_df['labels'].apply(lambda x: 'A' in x)]

In [19]:
muscular_degeneration_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename,old_filename
36753,50381,Unknown,Unknown,NaN,50381_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/001e39f3c4e7.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",50381_right.jpg,001e39f3c4e7.jpg
36754,50382,Unknown,Unknown,NaN,50382_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/0048f8f5f9c4.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",50382_right.jpg,0048f8f5f9c4.jpg
36755,50383,Unknown,Unknown,NaN,50383_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/00f483fe0d17.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",50383_right.jpg,00f483fe0d17.jpg
36756,50384,Unknown,Unknown,NaN,50384_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/01f94a77a3d5.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",50384_right.jpg,01f94a77a3d5.jpg
36757,50385,Unknown,Unknown,NaN,50385_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/024f9542aeeb.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",50385_right.jpg,024f9542aeeb.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38583,52211,Unknown,Unknown,NaN,52211_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/fe1df828dc1d.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",52211_right.jpg,fe1df828dc1d.jpg
38584,52212,Unknown,Unknown,NaN,52212_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/fe2e69065102.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",52212_right.jpg,fe2e69065102.jpg
38585,52213,Unknown,Unknown,NaN,52213_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/fe890b5a83b3.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",52213_right.jpg,fe890b5a83b3.jpg
38586,52214,Unknown,Unknown,NaN,52214_right.jpg,,macular degeneration,0,0,0,0,1,0,0,0,Training Images/ff458fc77afc.jpg,[A],"[0, 0, 0, 0, 1, 0, 0, 0]",52214_right.jpg,ff458fc77afc.jpg


In [24]:
new_df.head(5)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename,old_filename
0,13628,Unknown,Unknown,13628_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13628_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13628_left.jpg,0_left.jpg
1,13629,Unknown,Unknown,13629_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13629_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13629_left.jpg,103_left.jpg
2,13630,Unknown,Unknown,NaN,13630_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13630_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13630_right.jpg,1062_right.jpg
3,13631,Unknown,Unknown,13631_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13631_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13631_left.jpg,1083_left.jpg
4,13632,Unknown,Unknown,NaN,13632_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13632_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13632_right.jpg,1084_right.jpg


In [25]:
new_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename,old_filename
0,13628,Unknown,Unknown,13628_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13628_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13628_left.jpg,0_left.jpg
1,13629,Unknown,Unknown,13629_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13629_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13629_left.jpg,103_left.jpg
2,13630,Unknown,Unknown,NaN,13630_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13630_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13630_right.jpg,1062_right.jpg
3,13631,Unknown,Unknown,13631_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13631_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13631_left.jpg,1083_left.jpg
4,13632,Unknown,Unknown,NaN,13632_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13632_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13632_right.jpg,1084_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49124,62752,Unknown,Unknown,NaN,62752_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62752_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62752_right.jpg,Retinitis Pigmentosa95.jpg
49125,62753,Unknown,Unknown,NaN,62753_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62753_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62753_right.jpg,Retinitis Pigmentosa96.jpg
49126,62754,Unknown,Unknown,NaN,62754_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62754_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62754_right.jpg,Retinitis Pigmentosa97.jpg
49127,62755,Unknown,Unknown,NaN,62755_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62755_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62755_right.jpg,Retinitis Pigmentosa98.jpg


In [26]:
new_df.pop('old_filename')

0                        0_left.jpg
1                      103_left.jpg
2                    1062_right.jpg
3                     1083_left.jpg
4                    1084_right.jpg
                    ...            
49124    Retinitis Pigmentosa95.jpg
49125    Retinitis Pigmentosa96.jpg
49126    Retinitis Pigmentosa97.jpg
49127    Retinitis Pigmentosa98.jpg
49128    Retinitis Pigmentosa99.jpg
Name: old_filename, Length: 49129, dtype: object

In [27]:
new_df.head(5)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,13628,Unknown,Unknown,13628_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13628_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13628_left.jpg
1,13629,Unknown,Unknown,13629_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13629_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13629_left.jpg
2,13630,Unknown,Unknown,NaN,13630_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13630_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13630_right.jpg
3,13631,Unknown,Unknown,13631_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/13631_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13631_left.jpg
4,13632,Unknown,Unknown,NaN,13632_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/13632_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13632_right.jpg


In [28]:
full_df = pd.concat([existing_df, new_df], ignore_index=True)
full_df.to_csv('all_datasets2.csv', index=False)

In [29]:
full_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,Training Images/0_right.jpg,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,Training Images/1_right.jpg,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non diabetic retinopathy,moderate non diabetic retinopathy,0,1,0,0,0,0,0,1,Training Images/2_right.jpg,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild non diabetic retinopathy,0,1,0,0,0,0,0,1,Training Images/4_right.jpg,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non diabetic retinopathy,moderate non diabetic retinopathy,0,1,0,0,0,0,0,0,Training Images/5_right.jpg,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64643,62752,Unknown,Unknown,NaN,62752_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62752_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62752_right.jpg
64644,62753,Unknown,Unknown,NaN,62753_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62753_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62753_right.jpg
64645,62754,Unknown,Unknown,NaN,62754_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62754_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62754_right.jpg
64646,62755,Unknown,Unknown,NaN,62755_right.jpg,,other,0,0,0,0,0,0,0,1,Training Images/62755_right.jpg,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62755_right.jpg


In [14]:
filtered_df = new_df[new_df['labels'].apply(lambda x: 'C' in x)]

In [20]:
full_df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,Training Images/0_right.jpg,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,Training Images/1_right.jpg,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non diabetic retinopathy,moderate non diabetic retinopathy,0,1,0,0,0,0,0,1,Training Images/2_right.jpg,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild non diabetic retinopathy,0,1,0,0,0,0,0,1,Training Images/4_right.jpg,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non diabetic retinopathy,moderate non diabetic retinopathy,0,1,0,0,0,0,0,0,Training Images/5_right.jpg,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64643,62752,Unknown,Unknown,NaN,62752_right.jpg,,other,0,0,0,0,0,0,0,1,eye_diseases_classification\other/Retinitis Pi...,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62752_right.jpg
64644,62753,Unknown,Unknown,NaN,62753_right.jpg,,other,0,0,0,0,0,0,0,1,eye_diseases_classification\other/Retinitis Pi...,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62753_right.jpg
64645,62754,Unknown,Unknown,NaN,62754_right.jpg,,other,0,0,0,0,0,0,0,1,eye_diseases_classification\other/Retinitis Pi...,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62754_right.jpg
64646,62755,Unknown,Unknown,NaN,62755_right.jpg,,other,0,0,0,0,0,0,0,1,eye_diseases_classification\other/Retinitis Pi...,[O],"[0, 0, 0, 0, 0, 0, 0, 1]",62755_right.jpg


In [39]:
new_df.head(5)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename,old_filename
0,13628,Unknown,Unknown,13628_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/0_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13628_left.jpg,0_left.jpg
1,13629,Unknown,Unknown,13629_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/103_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13629_left.jpg,103_left.jpg
2,13630,Unknown,Unknown,NaN,13630_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/1062_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13630_right.jpg,1062_right.jpg
3,13631,Unknown,Unknown,13631_left.jpg,NaN,cataract,,0,0,0,1,0,0,0,0,Training Images/1083_left.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13631_left.jpg,1083_left.jpg
4,13632,Unknown,Unknown,NaN,13632_right.jpg,,cataract,0,0,0,1,0,0,0,0,Training Images/1084_right.jpg,[C],"[0, 0, 0, 1, 0, 0, 0, 0]",13632_right.jpg,1084_right.jpg
